# Автоматическая обработка языка
## НИУ ВШЭ, 2020-2021 учебный год

### Домашнее задание №2

Задание выполнил(а): Дарья Матяш

Ссылка на условия и требования: https://github.com/named-entity/hse-nlp/blob/master/hometask%202

1)В этой домашке вам будет нужно найти или самим написать русский и английский тексты (каждый от ста слов), в которых  будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их в ручную (а потом объяснить, какие моменты вы тут считаете трудными для автоматического посттеггинга и почему) – с помощью этих текстов мы будем оценивать качество работы наших теггоров. В текстах размечаем только части речи, ничего больше!
Вы получаете **балл** за **создание**, **разметку текста** и **объяснение** того, почему этот текст подходит для оценки (что в нём сложного). Всего за этот пункт **2 балла, т. к. языка 2**.



2)Потом вам будет нужно взять **три  POS теггера для русского (pymorphy2, mysteam, Natasha)** и **3  - для английского (SpyCy, Flair, NLTK)** и «прогнать» текст через каждый из них (если вы запустите только **2 теггера из трёх** – получите **балл**, если **три  из трёх** – **2 балла**, т. е. суммарно за этот пункт можно получить **4 балла**).


3)После этого вам надо будет оценить **accuracy** для каждого теггера. Заметьте, что в разных системах имена теггов и части речи  могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции или кода и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции. Этот пункт стоит **2 балла**.


Тут вы уже получили **8 баллов**.


4)Дальше вам нужно взять **лучший теггер** для русского языка и  с его помощью написать функцию, которая **повысит качество работы программы** из первой домашки. Так, многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Вам надо выделить **3 вида синтаксических групп** (к примеру не + какая-то часть речи или NP или сущ.+ наречие или еще что-то), запись которых в словарь, по вашему мнению, улучшила бы качество работы программы и создать такую функцию или функции, которые с помощью любых известных нам средств (chunking и regexp grammar, Natasha syntax parser,  код с последнего занятия по SpyCy, etc.) будет выделять эти группы в поданном в нее тексте. **Два балла** за саму **функцию**, **балл** за **объяснение** того, почему именно эти группы вы взяли.
***2 бонусных балла**, если **встроите эту функцию в программу из предыдущей домашки** и **сравните качества работы программы с нею и без неё**.

## 1) Создание текстов.

**Русский**: я написала сама небольшой текст, который состоит из разных штучек, которые могут быть сложными морфологическим парсерам: 
+ "стали" (гл vs. сущ.), 
+ "мимо" (нар. vs. предлог), 
+ "благодаря" (деепр. vs. предлог), 
+ "сорок" (числ. vs. сущ.), 
+ слова разных частей речи, чтобы посмотреть на работу в целом + всякие вещи, с которыми могут тоже возникнуть сложности при разметке ("нью-йорк", "мисо-суп", "черно-белый", "из-за", имя собственное - как поделит парсер?)

**Английский:** тоже я сама написала небольшой текст **тоже с небольшими проверками на омонимичные формы слов**:
+ "mine" (pron vs. noun vs. verb)
+ "use" (noun vs. verb)
+ "whole" (noun vs. adj.)
+ "while" (noun vs. conj.)
+ "please" (verb vs. adverb)
+ разные части речи для проверки


Теги я делала сама, старалась "упрощать" всякие местоимения-существительные/прилагательные в одну группу "местоимений", например, или причастие-деепричастие-инфинитив - это все глагол и тд.

In [415]:
import re

In [388]:
text_rus_list = ['Сегодня в Нью-Йорке хорошо, мы пошли в столовую и стали есть мисо-суп, салат и сорок пончиков.',
                 'Благодаря официанту мы узнали, что ложки из стали такие же прочные, как и из серебра.',
                 'Пять черно-белых сорок и желтых канареек за окном громко трещали.',
                 'На улице светило солнце, проезжали мимо нас машины, люди на велосипедах, а мы шли и фотографировали понравившиеся здания и памятники.',
                 'Ах, какая была отличная погода!',
                 'Пройдя два квартала, мы почувствовали усталость из-за тяжелых сумок, давящих на плечи.',
                 'Я захотела к себе домой, а моя подруга пригласила в свой дом на чашку кофе.',
                 'Оля, моя подруга, сварила отличный кофе, но он мне не понравился.',
                 'Потом мы быстро собрались и продолжили прогулку снова, благодаря солнце за погоду.']

In [389]:
rus_text_sents = []
for sent in text_rus_list:
    rus_text_sents.append(' '.join([re.sub('[|\.|\?|!|,|\(|\)]','',w.lower()) for w in sent.split()]))
        

In [390]:
rus_text_tags_list = ['ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB NOUN NOUN CONJ NUM NOUN',
                      'PREP NOUN PRON VERB CONJ NOUN PREP NOUN PRON PART ADJ CONJ CONJ PREP NOUN',
                      'NUM ADJ NOUN CONJ ADJ NOUN PREP NOUN ADV VERB',
                      'PREP NOUN VERB NOUN VERB PREP PRON NOUN NOUN PREP NOUN CONJ PRON VERB CONJ VERB VERB NOUN CONJ NOUN',
                      'INTJ PRON VERB ADJ NOUN',
                      'VERB NUM NOUN PRON VERB NOUN PREP ADJ NOUN VERB PREP NOUN',
                      'PRON VERB PREP PRON ADV CONJ PRON NOUN VERB PREP PRON NOUN PREP NOUN NOUN',
                      'NOUN PRON NOUN VERB ADJ NOUN CONJ PRON PRON PART VERB',
                      'ADV PRON ADV VERB CONJ VERB NOUN ADV VERB NOUN PREP NOUN']

In [391]:
text_eng_list = ['I use my shorts, T-shirts and shirts for flirt.',
                'The use of short sentences is popular nowadays.',
                'Being rude is so rude.',
                'Have you done your homework?',
                 'I did it yesterday.',
                'I have just done it.',
                'I am doing it now.',
                'Please, be mine!',
                'I mine a whole on Fridays.',
                'I saw a mine and I was scared.',
                'Have you seen that mine shaft?',
                'There is no other evidence of that in the whole world.',
                'While you were doing your homework I made myself a monkey costume.',
                'Please wait for a while!',
                 'I want to please you',
                'Have you seen three monkeys in the zoo?']

In [392]:
eng_text_sents = []
for sent in text_eng_list:
    eng_text_sents.append(' '.join([re.sub('[|\.|\?|!|,|\(|\)]','',w.lower()) for w in sent.split()]))

In [393]:
eng_text_tags_list = ['PRON VERB PRON NOUN NOUN CONJ NOUN ADP NOUN',
                      'DET NOUN ADP ADJ NOUN VERB ADJ ADV',
                      'VERB NOUN VERB ADV ADJ',
                      'VERB PRON VERB PRON NOUN',
                      'PRON VERB PRON NOUN',
                      'PRON VERB ADV VERB PRON',
                      'PRON VERB VERB PRON ADV',
                      'ADV VERB PRON',
                      'PRON VERB DET NOUN ADP NOUN',
                      'PRON VERB DET NOUN CONJ NOUN VERB VERB',
                      'VERB PRON VERB ADP NOUN NOUN',
                      'DET VERB DET ADJ NOUN ADP DET ADP DET ADJ NOUN',
                      'CONJ PRON VERB VERB PRON NOUN PRON VERB PRON DET NOUN NOUN',
                      'ADV VERB ADP DET NOUN',
                      'PRON VERB PRT VERB PRON',
                      'VERB PRON VERB NUM NOUN ADP NUM NOUN']

In [394]:
tag_lists = {'rus_text_tags_list': rus_text_tags_list,'eng_text_tags_list':eng_text_tags_list}

## 3) Функции для оценки accuracy.

Мне проще сделать функцию здесь и "по ходу пьесы" смотреть accuracy каждого морфологического парсера)
Я считаю максимальную схожесть, и не считаю проблемой, если какое-то слово поделено как-то не так или на два слова, я смотрю "по максимуму"
В скобках приведена функция, в которой я доходила до места несоотвествия и "впадала в ступор", мне так не понравилось, поэтому я сделала по-другому:)

In [401]:
# def acc(x, lang):
#     tags_list = tag_lists['{0}_text_tags_list'.format(lang)]
#     ans = 0
#     for sent_ind in range(len(tags_list)):
#         sent = x[sent_ind]
#         if len(sent.split()) == len(tags_list[sent_ind].split()):
#             sent_acc = 0
#             for i in range(len(sent.split())):
#                 if sent.split()[i] == tags_list[sent_ind].split()[i]:
#                     sent_acc += 1
# #             print(sent_acc/len(sent.split()))
#             ans += sent_acc/len(sent.split())
#         else:
#             if len(sent.split()) < len(tags_list[sent_ind].split()):
#                 sent_acc = 0
#                 for i in range(len(sent.split())):
#                     if sent.split()[i] == tags_list[sent_ind].split()[i]:
#                         sent_acc +=1
#                     else:
#                         print('1',sent_ind, sent, tags_list[sent_ind].split()[i:], sent.split()[i:])
#             else:
#                 sent_acc = 0
#                 for i in range(len(tags_list[sent_ind].split())):
#                     if sent.split()[i] == tags_list[sent_ind].split()[i]:
#                         sent_acc +=1
#                     else:
#                         print('2', sent_ind, sent, tags_list[sent_ind].split()[i:], sent.split()[i:])
#     return ans/len(tags_list)

In [685]:
def acc(x, lang):
    tags_list = tag_lists['{0}_text_tags_list'.format(lang)]
    ans = 0
    for sent_ind in range(len(tags_list)):
        A = x[sent_ind]
        B = tags_list[sent_ind]
        n = len(A)
        m = len(B)
        F = [[0] * (m + 1) for i in range(n + 1)]
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                if A[i - 1] == B[j - 1]:
                    F[i][j] = F[i - 1][j - 1] + 1
                else:
                    F[i][j] = max(F[i - 1][j], F[i][j - 1])
        Ans = []
        i = n
        j = m
        while i > 0 and j > 0:
            if A[i - 1] == B[j - 1]:
                Ans.append(A[i - 1])
                i -= 1
                j -= 1
            elif F[i - 1][j] == F[i][j]:
                i -= 1
            else:
                j -= 1
        Ans = Ans[::-1]
        ans += len(''.join(Ans).split())/ len(x[sent_ind].split())
    return ans / len(tags_list)

## 2) Морфологические парсеры

### 2.1) Mystem

In [396]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

In [397]:
mystem_tags_dict = {'A': 'ADJ',
                    'ADV': 'ADV',
                    'ADVPRO': 'ADV',
                    'ANUM': 'ADJ',
                    'APRO': 'ADV',
                    'COM': 'ADJ',
                    'SCONJ': 'CONJ',
                    'CONJ': 'CONJ',
                    'INTJ': 'INTJ',
                    'NONLEX': 'X',
                    'NUM': 'NUM',
                    'PART': 'PART',
                    'PR': 'PREP',
                    'S': 'NOUN',
                    'SPRO': 'PRON',
                    'UNKN': 'X',
                    'V': 'VERB'}

In [398]:
mystem_tags = []
for sent in rus_text_sents:
    ans = ''
    for a in mystem_analyzer.analyze(sent):
        if 'analysis' in a:
            if 'gr' in a['analysis'][0].keys():
                gr = a['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                ans += mystem_tags_dict[pos] + ' '
    mystem_tags.append(ans[:-1])
                

In [399]:
for s in range(len(mystem_tags)):
    print(s, ') ', rus_text_sents[s], '\n', '   ', mystem_tags[s],'\n', '   ', rus_text_tags_list[s])

0 )  сегодня в нью-йорке хорошо мы пошли в столовую и стали есть мисо-суп салат и сорок пончиков 
     ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB NOUN NOUN NOUN CONJ NUM NOUN 
     ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB NOUN NOUN CONJ NUM NOUN
1 )  благодаря официанту мы узнали что ложки из стали такие же прочные как и из серебра 
     PREP NOUN PRON VERB CONJ NOUN PREP VERB ADV PART ADJ CONJ CONJ PREP NOUN 
     PREP NOUN PRON VERB CONJ NOUN PREP NOUN PRON PART ADJ CONJ CONJ PREP NOUN
2 )  пять черно-белых сорок и желтых канареек за окном громко трещали 
     NUM ADJ NUM CONJ ADJ NOUN PREP NOUN ADV VERB 
     NUM ADJ NOUN CONJ ADJ NOUN PREP NOUN ADV VERB
3 )  на улице светило солнце проезжали мимо нас машины люди на велосипедах а мы шли и фотографировали понравившиеся здания и памятники 
     PREP NOUN VERB NOUN VERB PREP PRON NOUN NOUN PREP NOUN CONJ PRON VERB CONJ VERB VERB NOUN CONJ NOUN 
     PREP NOUN VERB NOUN VERB PREP PRON NOUN NOUN PREP NOUN CONJ PRON 

In [686]:
acc(mystem_tags, 'rus')

0.929659338482868

### 2.2) PYMORPHY2

In [208]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [341]:
pymorphy_tags_dict = {
    'NOUN': 'NOUN',
    'VERB': 'VERB',
    'PRTF': 'VERB',
    'GRND': 'VERB',
    'ADJF': 'ADJ',
    'ADJS': 'ADJ',
    'PREP': 'PREP',
    'SPRO': 'PRON',
    'APRO': 'PRON',
    'NPRO': 'PRON',
    'ADVPRO': 'PRON',
    'ADVB': 'ADV',
    'CONJ': 'CONJ',
    'NUMR': 'NUM',
    'PRCL': 'PART',
    'INTJ': 'INTJ'
}

In [404]:
pymorphy_tags = []
for sent in rus_text_sents:
    ans = ''
    for word in sent.split():
        p = str(m.parse(word)[0].tag)
        ans += pymorphy_tags_dict[re.findall('[A-Z]{1,5}', p)[0]] + ' '
    pymorphy_tags.append(ans[:-1])

In [405]:
for s in range(len(pymorphy_tags)):
    print(s, ') ', rus_text_sents[s], '\n', '   ', pymorphy_tags[s],'\n', '   ', rus_text_tags_list[s])

0 )  сегодня в нью-йорке хорошо мы пошли в столовую и стали есть мисо-суп салат и сорок пончиков 
     ADV PREP NOUN ADV PRON VERB PREP ADJ CONJ VERB INTJ NOUN NOUN CONJ NUM NOUN 
     ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB NOUN NOUN CONJ NUM NOUN
1 )  благодаря официанту мы узнали что ложки из стали такие же прочные как и из серебра 
     PREP NOUN PRON VERB CONJ NOUN PREP VERB ADJ PART ADJ CONJ CONJ PREP NOUN 
     PREP NOUN PRON VERB CONJ NOUN PREP NOUN PRON PART ADJ CONJ CONJ PREP NOUN
2 )  пять черно-белых сорок и желтых канареек за окном громко трещали 
     NUM ADJ NUM CONJ ADJ NOUN PREP NOUN ADV VERB 
     NUM ADJ NOUN CONJ ADJ NOUN PREP NOUN ADV VERB
3 )  на улице светило солнце проезжали мимо нас машины люди на велосипедах а мы шли и фотографировали понравившиеся здания и памятники 
     PREP NOUN VERB NOUN VERB ADV PRON NOUN NOUN PREP NOUN CONJ PRON VERB CONJ VERB VERB NOUN CONJ NOUN 
     PREP NOUN VERB NOUN VERB PREP PRON NOUN NOUN PREP NOUN CONJ PRON VERB CO

In [687]:
acc(pymorphy_tags, 'rus')

0.9167508417508416

### 2.3) NATASHA

In [241]:
natasha_tags_dict = {
    'NOUN': 'NOUN',
    'VERB': 'VERB',
    'AUX': 'VERB',
    'ADJ': 'ADJ',
    'PREP': 'PREP',
    'PRON': 'PRON',
    'DET': 'PRON',
    'ADV': 'ADV',
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ',
    'NUM': 'NUM',
    'PART': 'PART',
    'ADP':'PREP'
}

In [242]:
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc)

In [243]:
morph_vocab = MorphVocab()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [407]:
natasha_tags = []
for sent in rus_text_sents:
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    natasha_tags.append(' '.join([ natasha_tags_dict[_.pos] for _ in doc.tokens]))

In [408]:
for s in range(len(natasha_tags)):
    print(s, ') ', rus_text_sents[s], '\n', '   ', natasha_tags[s],'\n', '   ', rus_text_tags_list[s])

0 )  сегодня в нью-йорке хорошо мы пошли в столовую и стали есть мисо-суп салат и сорок пончиков 
     ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB ADJ NOUN CONJ NUM NOUN 
     ADV PREP NOUN ADV PRON VERB PREP NOUN CONJ VERB VERB NOUN NOUN CONJ NUM NOUN
1 )  благодаря официанту мы узнали что ложки из стали такие же прочные как и из серебра 
     PREP NOUN PRON VERB CONJ NOUN PREP VERB PRON PART ADJ CONJ PART PREP NOUN 
     PREP NOUN PRON VERB CONJ NOUN PREP NOUN PRON PART ADJ CONJ CONJ PREP NOUN
2 )  пять черно-белых сорок и желтых канареек за окном громко трещали 
     NUM ADJ NOUN CONJ ADJ NOUN PREP NOUN ADV VERB 
     NUM ADJ NOUN CONJ ADJ NOUN PREP NOUN ADV VERB
3 )  на улице светило солнце проезжали мимо нас машины люди на велосипедах а мы шли и фотографировали понравившиеся здания и памятники 
     PREP NOUN VERB NOUN VERB ADV PRON NOUN NOUN PREP NOUN CONJ PRON VERB CONJ VERB ADJ NOUN CONJ NOUN 
     PREP NOUN VERB NOUN VERB PREP PRON NOUN NOUN PREP NOUN CONJ PRON VERB C

In [688]:
acc(natasha_tags, 'rus')

0.9477693602693604

### 2.4) SPACY

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [382]:
spacy_tags_dict = {
    'NOUN': 'NOUN',
    'VERB': 'VERB',
    'AUX': 'VERB',
    'ADJ': 'ADJ',
    'PREP': 'PREP',
    'PRON': 'PRON',
    'DET': 'DET',
    'ADV': 'ADV',
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ',
    'NUM': 'NUM',
    'PART': 'PRT',
    'ADP':'VERB',
    'PUNCT':'',
    'INTJ': 'ADV'
    
}

In [383]:
spacy_tags = []
for sent in eng_text_sents:
    for s in nlp(sent).sents:
        ans = ''
        for t in s:
            ans += spacy_tags_dict[t.pos_] + ' '
        spacy_tags.append(ans[:-1])

In [384]:
for s in range(len(spacy_tags)):
    print(s, ') ', eng_text_sents[s], '\n', '   ', spacy_tags[s],'\n', '   ', eng_text_tags_list[s])

0 )  i use my shorts t-shirts and shirts for flirt 
     PRON VERB DET NOUN NOUN  NOUN CONJ NOUN VERB NOUN 
     PRON VERB PRON NOUN NOUN CONJ NOUN ADP NOUN
1 )  the use of short sentences is popular nowadays 
     DET NOUN VERB ADJ NOUN VERB ADJ ADV 
     DET NOUN ADP ADJ NOUN VERB ADJ ADV
2 )  being rude is so rude 
     VERB ADJ VERB ADV ADJ 
     VERB NOUN VERB ADV ADJ
3 )  have you done your homework 
     VERB PRON VERB DET NOUN 
     VERB PRON VERB PRON NOUN
4 )  i did it yesterday 
     PRON VERB PRON NOUN 
     PRON VERB PRON NOUN
5 )  i have just done it 
     PRON VERB ADV VERB PRON 
     PRON VERB ADV VERB PRON
6 )  i am doing it now 
     PRON VERB VERB PRON ADV 
     PRON VERB VERB PRON ADV
7 )  please be mine 
     ADV VERB PRON 
     ADV VERB PRON
8 )  i mine a whole on fridays 
     PRON VERB DET NOUN VERB NOUN 
     PRON VERB DET NOUN ADP NOUN
9 )  i saw a mine and i was scared 
     PRON VERB DET NOUN CONJ PRON VERB ADJ 
     PRON VERB DET NOUN CONJ NOUN VERB VERB
10

In [689]:
print(acc(spacy_tags,'eng'))

0.8860795454545455


### 2.5) FLAIR

In [38]:
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

In [379]:
flair_tags_dict = {
    'NNS': 'NOUN',
    'NN': 'NOUN',
    'VB': 'VERB',
    'VBP': 'VERB',
    'VBZ': 'VERB',
    'VBN': 'VERB',
    'VBD': 'VERB',
    'VBG': 'VERB',
    'DT': 'DET',
    'EX': 'DET',
    'JJ': 'ADJ',
    'IN': 'CONJ',
    'PRP': 'PRON',
    'PRP$': 'PRON',
    'RB': 'ADV',
    'UH': 'ADV',
    'CC': 'CONJ',
    'SCONJ': 'CONJ',
    'CD': 'NUM',
    'PART': 'PART',
    'INN':'ADP',
    'TO': 'PRT'
}

In [380]:
flair_tags = []
for sent in eng_text_sents:
    sentence = Sentence(sent)
    tagger.predict(sentence)
    ans = ''
    for tags in re.findall('<(.*?)>',sentence.to_tagged_string()):
        ans += flair_tags_dict[tags] + ' '
    flair_tags.append(ans[:-1])

In [411]:
for s in range(len(flair_tags)):
    print(s, ') ', eng_text_sents[s], '\n', '   ', flair_tags[s],'\n', '   ', eng_text_tags_list[s])

0 )  i use my shorts t-shirts and shirts for flirt 
     PRON VERB PRON NOUN NOUN CONJ NOUN CONJ NOUN 
     PRON VERB PRON NOUN NOUN CONJ NOUN ADP NOUN
1 )  the use of short sentences is popular nowadays 
     DET NOUN CONJ ADJ NOUN VERB ADJ ADV 
     DET NOUN ADP ADJ NOUN VERB ADJ ADV
2 )  being rude is so rude 
     VERB ADJ VERB ADV ADJ 
     VERB NOUN VERB ADV ADJ
3 )  have you done your homework 
     VERB PRON VERB PRON NOUN 
     VERB PRON VERB PRON NOUN
4 )  i did it yesterday 
     PRON VERB PRON NOUN 
     PRON VERB PRON NOUN
5 )  i have just done it 
     PRON VERB ADV VERB PRON 
     PRON VERB ADV VERB PRON
6 )  i am doing it now 
     PRON VERB VERB PRON ADV 
     PRON VERB VERB PRON ADV
7 )  please be mine 
     ADV VERB PRON 
     ADV VERB PRON
8 )  i mine a whole on fridays 
     PRON VERB DET NOUN CONJ NOUN 
     PRON VERB DET NOUN ADP NOUN
9 )  i saw a mine and i was scared 
     PRON VERB DET NOUN CONJ PRON VERB ADJ 
     PRON VERB DET NOUN CONJ NOUN VERB VERB
10 )  

In [690]:
print(acc(flair_tags, 'eng'))

0.9228377525252526


### 2.6) NLTK

In [53]:
import nltk
from nltk import WordNetLemmatizer

wnl = WordNetLemmatizer()

In [377]:
nltk_tags = []
for i in range(len(eng_text_sents)):
    tokens=nltk.word_tokenize(eng_text_sents[i])
    ans = ''
    for tup in nltk.pos_tag(tokens, tagset='universal'):
        ans += tup[1] + ' '
    nltk_tags.append(ans[:-1])

In [378]:
for s in range(len(nltk_tags)):
    print(s, ') ', eng_text_sents[s], '\n', '   ', nltk_tags[s],'\n', '   ', eng_text_tags_list[s])

0 )  i use my shorts t-shirts and shirts for flirt 
     NOUN VERB PRON NOUN NOUN CONJ NOUN ADP NOUN 
     PRON VERB PRON NOUN NOUN CONJ NOUN ADP NOUN
1 )  the use of short sentences is popular nowadays 
     DET NOUN ADP ADJ NOUN VERB ADJ NOUN 
     DET NOUN ADP ADJ NOUN VERB ADJ ADV
2 )  being rude is so rude 
     VERB NOUN VERB ADV ADJ 
     VERB NOUN VERB ADV ADJ
3 )  have you done your homework 
     VERB PRON VERB PRON NOUN 
     VERB PRON VERB PRON NOUN
4 )  i did it yesterday 
     NOUN VERB PRON NOUN 
     PRON VERB PRON NOUN
5 )  i have just done it 
     NOUN VERB ADV VERB PRON 
     PRON VERB ADV VERB PRON
6 )  i am doing it now 
     NOUN VERB VERB PRON ADV 
     PRON VERB VERB PRON ADV
7 )  please be mine 
     NOUN VERB ADJ 
     ADV VERB PRON
8 )  i mine a whole on fridays 
     NOUN VERB DET NOUN ADP NOUN 
     PRON VERB DET NOUN ADP NOUN
9 )  i saw a mine and i was scared 
     NOUN VERB DET NOUN CONJ NOUN VERB VERB 
     PRON VERB DET NOUN CONJ NOUN VERB VERB
10 )  

In [691]:
print(acc(nltk_tags, 'eng'))

0.9249999999999999


# 4. Лучший теггер + прошлая домашка

Как прекрасно сохранять все в базу данных, открываем бд с прошлой домашки, пишем одну функцию и сдаем домашку

In [632]:
import pandas as pd
df = pd.read_csv('NLP-HW1.csv', encoding='utf-8')

Моя идея очень простая: с помощью лучшего теггера (им оказался natasha) я найду синтаксические группы (ниже описание и причина выбора), которые просто доставлю в конец лемм, чтобы их как бы сделать "тяжелее" как параметр позитивного/негативного отзыва. Мне кажется это логичным, тк мы типа на момент этой домашки ничего особо не знали про tf-idf, а идея с "большим весом" логичная - что чаще встречается, что синтаксически важнее, то и больше учитываем при выборе сентимента
Как синтаксически группы?
+ "**существительное + прилагательное**" - потому что часто в отзывах мы смотрим на характеристики чего-либо (например, "Это **отличное заведение**!" или "**Ужасный сервис** и **холодные блюда**" - и сразу все понятно
+ "не" + глагол - здесь тоже понятно, хочется думать про всякие вещи типа "Мне **не понравилось/ не хочется** приходить еще" и тд
+ "**существительное + существительное**" - тут хочется искать всякие "выбор игрушек", "поведение официанта", чтобы понимать фокус отзыва - как вообще человек оценивал заведение/место

Я не гарантирую, что все это сработает или идеально будет находить такое, но можно попробовать:)

In [633]:
from natasha import NewsSyntaxParser
syntax_parser = NewsSyntaxParser(emb)


def syntax(sent):
    ans = []
    adj_noun_list = []
    np_list = []
    no_verb_list = []
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for i in range(len(doc.tokens)):
        np = ''
        if list(doc.tokens[i])[6] == 'ADJ':
            head = int(list(doc.tokens[i])[4].split('_')[-1]) - 1
            if list(doc.tokens[head])[6] == 'NOUN':
                adj_noun_list.append(list(doc.tokens[i])[2] + ' ' + list(doc.tokens[head])[2])
        elif list(doc.tokens[i])[2] == 'не':
            head = int(list(doc.tokens[i])[4].split('_')[-1]) - 1
            if list(doc.tokens[head])[6] == 'VERB':
                no_verb_list.append('не ' + list(doc.tokens[head])[2])
        elif list(doc.tokens[i])[6] == 'NOUN':
            head = int(list(doc.tokens[i])[4][-1]) - 1
            if list(doc.tokens[head])[6] == 'NOUN':
                np_list.append(list(doc.tokens[i])[2] + ' ' + list(doc.tokens[head])[2])
    ans.extend(adj_noun_list)
    ans.extend(np_list)
    ans.extend(no_verb_list)
    return sent + ' ' + ' '.join(ans)

In [634]:
df['l_s'] = df['lemmas'].apply(lambda x: syntax(x))

Сделали, теперь хорошо бы импортировать функции из прошлой домашки, но, честно говоря, мне проще руками скопировать несложный код, хотя я понимаю, что так не очыень круто 

In [662]:
#тестовая выборка
pos_rev_lem_list = df[(df['rates']==5)]['l_s'].to_list()[:-10]
neg_rev_lem_list = df[(df['rates']==1)]['l_s'].to_list()[:-10]

#словари со словами, встречающихся в только положительных или только отрицательных отзывах
to_string_pos = ''
for el in pos_rev_lem_list:
    to_string_pos += ' ' + el
to_string_neg = ''
for el in neg_rev_lem_list:
    to_string_neg += ' ' + el
    
from collections import Counter
pos_rev_c = Counter(to_string_pos.split())
neg_rev_c = Counter(to_string_neg.split())

pos_rev_set = set([tup[0] for tup in pos_rev_c.most_common() if 25 > tup[1] > 1 and tup[0].isalpha()])
neg_rev_set = set([tup[0] for tup in neg_rev_c.most_common() if 50 > tup[1] > 1 and tup[0].isalpha()])

only_pos = pos_rev_set - pos_rev_set.intersection(neg_rev_set)
only_neg = neg_rev_set - pos_rev_set.intersection(neg_rev_set)

#функция для определения сентимента
def which_sent(x):
    pos_sent = 0
    neg_sent = 0
    for word in x.split():
        if word in only_pos:
            pos_sent +=1
        elif word in only_neg:
            neg_sent +=1
        else:
            pass
    if pos_sent > neg_sent:
        return 5
    else:
        return 1
#Тестовая выборка
X_test = df[(df['rates']==5)]['l_s'].to_list()[-10:] + df[(df['rates']==1)]['l_s'].to_list()[-10:]
y_test = df[(df['rates']==5)]['rates'].to_list()[-10:] + df[(df['rates']==1)]['rates'].to_list()[-10:]

#подсчет accuracy
def count_accuracy(X, y):
    ans = 0
    for i in range(len(X)):
        if which_sent(X[i]) == y[i]:
            ans += 1
    return ans/len(y)

print('accuracy на тестовой выборке: ', count_accuracy(X_test, y_test) )

accuracy на тестовой выборке:  0.55


Ого, отлично, стало лучше (по сравнению с прошлым разом)!!

Но все еще некруто, хочу отдельно взять те найденные синтаксические группы и добавить их отдельно в словарь!!!

In [636]:
def syntax2(sent):
    ans = []
    adj_noun_list = []
    np_list = []
    no_verb_list = []
    doc = Doc(sent)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for i in range(len(doc.tokens)):
        np = ''
        if list(doc.tokens[i])[6] == 'ADJ':
            head = int(list(doc.tokens[i])[4].split('_')[-1]) - 1
            if list(doc.tokens[head])[6] == 'NOUN':
                adj_noun_list.append(list(doc.tokens[i])[2] + ' ' + list(doc.tokens[head])[2])
        elif list(doc.tokens[i])[2] == 'не':
            head = int(list(doc.tokens[i])[4].split('_')[-1]) - 1
            if list(doc.tokens[head])[6] == 'VERB':
                no_verb_list.append('не ' + list(doc.tokens[head])[2])
        elif list(doc.tokens[i])[6] == 'NOUN':
            head = int(list(doc.tokens[i])[4][-1]) - 1
            if list(doc.tokens[head])[6] == 'NOUN':
                np_list.append(list(doc.tokens[i])[2] + ' ' + list(doc.tokens[head])[2])
    ans.extend(adj_noun_list)
    ans.extend(np_list)
    ans.extend(no_verb_list)
    return ans

In [637]:
pos_rev_l = []
neg_rev_l = []
for i in range(len(df['lemmas'].to_list())):
    if df['rates'].to_list()[i] == 5:
        pos_rev_l.extend(syntax2(df['lemmas'].to_list()[i]))
    else:
        neg_rev_l.extend(syntax2(df['lemmas'].to_list()[i]))

pos_rev_set = pos_rev_set.union(set(pos_rev_l))
neg_rev_set = neg_rev_set.union(set(neg_rev_l))

# for i in range(len(df['lemmas'].to_list())):
#     if df['rates'].to_list()[i] == 5:
#         pos_rev_set.union(syntax2(df['lemmas'].to_list()[i]))
#     else:
#         neg_rev_set.union(syntax2(df['lemmas'].to_list()[i]))

In [656]:
only_pos = pos_rev_set - pos_rev_set.intersection(neg_rev_set)
only_neg = neg_rev_set - pos_rev_set.intersection(neg_rev_set)

#функция для определения сентимента
def which_sent2(x):
    pos_sent = 0
    neg_sent = 0
    l = x.split()
    for i in range(len(l)-1):
        if l[i] in only_pos:
            pos_sent +=1
        elif l[i] in only_neg:
            neg_sent +=1
        elif l[i] + ' ' + l[i+1] in only_pos:
            pos_sent +=1 
        elif l[i] + ' ' + l[i+1] in only_neg:
            neg_sent +=1    
    if pos_sent > neg_sent:
        return 5
    else:
        return 1
                   
                
#Тестовая выборка
X_test = df[(df['rates']==5)]['lemmas'].to_list()[-10:] + df[(df['rates']==1)]['lemmas'].to_list()[-10:]
y_test = df[(df['rates']==5)]['rates'].to_list()[-10:] + df[(df['rates']==1)]['rates'].to_list()[-10:]


#подсчет accuracy
def count_accuracy(X, y):
    ans = 0
    for i in range(len(X)):
        if which_sent2(X[i]) == y[i]:
            ans += 1
    return ans/len(y)

print('accuracy на тестовой выборке: ', count_accuracy(X_test, y_test) )

accuracy на тестовой выборке:  0.9
